# 讀取資料

In [ ]:
from dataManager import getData, getStockCodeList

In [ ]:
getStockCodeList()

In [ ]:
target_data = getData('M2500')

# 資料視覺化

In [ ]:
import matplotlib.pyplot as plt

#### 收盤價

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(target_data['收盤價(元)'])
plt.title('close pirce')
plt.xlabel('Time')
plt.ylabel('pirce')
plt.show()

#### 成交量

In [ ]:
plt.figure(figsize=(20, 8))
plt.bar(target_data.index, target_data['成交量(千股)'])
plt.title('volume')
plt.xlabel('Time')
plt.ylabel('volume')
plt.show()

#  定態 or 非定態 - 單根檢定

In [ ]:
test_data = target_data['ret']
# test_data = target_data['收盤價(元)']

## ADF - 假設:  資料有單根(資料非定態)

In [ ]:
# !pip install arch==4.15

In [ ]:
from arch.unitroot import ADF

In [ ]:
# ADF?

In [ ]:
# 假設:  資料有單根(資料非定態)
adf_test = ADF(test_data, max_lags=10 ,trend="c",method="BIC")
adf_test

## PP - 假設:  資料有單根(資料非定態)

In [ ]:
from arch.unitroot import PhillipsPerron

# 假設:  資料有單根(資料非定態)
pp_test = PhillipsPerron(test_data)
pp_test

## KPSS - 假設:  資料是弱定態

In [ ]:
from arch.unitroot import KPSS 

# 假設:  資料是弱定態
kpss_test = KPSS(test_data)
kpss_test

# 加分題：單根檢定多數決

In [ ]:
from arch.unitroot import KPSS 
from arch.unitroot import PhillipsPerron
from arch.unitroot import ADF


def isUnitRoot(series_data):
    # 假設:  資料有單根(資料非定態)
    adf_test = ADF(series_data, max_lags=10 ,trend="c",method="BIC")
    pp_test = PhillipsPerron(series_data)
    
    # 假設:  資料是弱定態
    kpss_test = KPSS(series_data)
    return sum([ adf_test.pvalue > 0.1, pp_test.pvalue > 0.1, kpss_test.pvalue <= 0.1]) > 2

In [ ]:
isUnitRoot(test_data)

# 白噪音檢定(純隨機性)
#### 資料有沒有自我相關

### ACF 自我相關 && PACF 偏自我相關

In [ ]:
# !pip install statsmodels==0.11.1    

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

plot_acf(test_data, ax=ax, lags=30, alpha=0.05) 
plot_pacf(test_data, ax=ax2, lags=30, alpha=0.05) 
ax.grid()
ax2.grid()
plt.show()

In [ ]:
# acf?

In [ ]:
## 連第0期也有呈現
acf(test_data, nlags=30, alpha=0.05) 

In [ ]:
pacf(test_data, nlags=30, alpha=0.05) 

### Ljung-Box Q檢定

<p>The Ljung-Box test is a statistical test that checks if autocorrelation exists in a time series.</p>
<hr>
<br>H0: 這一資料t是獨立的</br>
<br>HA: 這一資料t是非獨立，有存在自我相關</br>

In [ ]:
# ?acf

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
acf_values, confidence ,q_values, p_values = acf(test_data, nlags=30, alpha=0.05, qstat = True) 

In [ ]:
acf_values

In [ ]:
confidence

In [ ]:
q_values

In [ ]:
p_values

#### 另一種 Ljung-Box Q檢定 方式

In [ ]:
import statsmodels.stats.diagnostic as diag

In [ ]:
# ?diag.acorr_ljungbox

In [ ]:
## return ( q values, p values)
diag.acorr_ljungbox(
    test_data, 
    lags=30,
)

In [ ]:
## return ( Ljung-Box 檢定值 , 依照卡方分配（k=40）的 p values)
diag.acorr_ljungbox(
    test_data, 
    lags=[30],
)

In [ ]:
q_values, p_values = diag.acorr_ljungbox(
    test_data, 
    lags=[30],
)
print('可以有{}百分比的把握說此序列不是純白噪音'.format(1-p_values))